In [2]:
import schedule
import pandas as pd
import time
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Load your dataset
data = pd.read_csv('../../data/price_2.csv')

# Define a function to train the model
def train_model():
    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(data.drop('y', axis=1), data['y'], test_size=0.2, random_state=42)
    
    # Train the model
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    
    # Evaluate the model
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Model trained with accuracy: {accuracy:.4f}")

# Schedule the task to run every day at 2am
schedule.every(1).day.at("02:00").do(train_model)  # run every day at 2am

while True:
    schedule.run_pending()
    time.sleep(1)